In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import numpy as np
from pykrx import stock
import math
from random import *
import random
def labellingD0(d0) -> str:
    '''
    D0 시점의 각 봉에 대한 라벨링 25가지
    '''
    openP = d0['open']
    highP = d0['high']
    lowP = d0['low']
    closeP = d0['close']
    
    # 장대 양봉
    if closeP >= 1.1*openP:
        if closeP < highP:
            if openP > lowP:
                return "P15"
            if highP - 2*closeP + openP >= 0:
                return "P14"
            return "P13"
        if openP > lowP:
            return "P11"
        return "P10"

    # 짧은 양봉
    elif closeP >= 1.005*openP:
        if closeP < highP:
            if openP > lowP:
                return "P05"
            if highP -3*closeP + 2*openP >= 0:
                return "P04"
            return "P03"
        if openP > lowP:
            if 2*highP - 3*openP + lowP >= 0:
                return "P01"
            return "P02"
        return "P00"

    # 보합
    elif closeP >= openP:
        end_min = closeP - lowP
        Max_end = highP - closeP
        if end_min > Max_end*3:
            return "K01"
        elif end_min*3 < Max_end:
            return "K02"
        return "K00"

    # 짧은 음봉
    elif closeP >= 0.9*openP:
        if openP < highP :
            if closeP > lowP :
                return "M05"
            if highP -3*openP +2*closeP < 0:
                return "M03"
            return "M04"
        if closeP > lowP :
            if 3*closeP - lowP - 2*openP < 0:
                return "M01"
            return "M02"
        return "M00"

    # 장대 음봉
    else:
        if closeP > lowP :
            if openP < highP :
                return "M15"
            return "M11"
        if openP < highP :
            if highP -2*openP + closeP >= 0:
                return "M14"
            return "M13"
        return "M10"


def labellingD1(d10) -> str:
    '''
    D1 5가지 x D0 25가지
    '''
    temp = d10.iloc[0]
    openP = temp['open']
    highP = temp['high']
    lowP = temp['low']
    closeP = temp['close']
    
    # 장대 양봉
    if closeP >= 1.1*openP:
        res = "P10"

    # 짧은 양봉
    elif closeP >= 1.005*openP:
        res = "P00"

    # 보합
    elif closeP >= openP:
        res = "K00"

    # 짧은 음봉
    elif closeP >= 0.9*openP:
        res = "M00"

    # 장대 음봉
    else:
        res = "M10"

    return res + labellingD0(d10.iloc[1])

def labellingD2(d210):
    d2_openP = d210.iloc[0]['open']
    d2_closeP = d210.iloc[0]['close']
    d1_openP = d210.iloc[1]['open']
    d1_closeP = d210.iloc[1]['close']
    
    d21_max = max(d2_openP, d2_closeP, d1_openP, d1_closeP)
    d21_avg = (d21_max + min(d2_openP, d2_closeP, d1_openP, d1_closeP))/2
    if d21_max/d21_avg <= 1.005:
        res = "S04"
    elif d2_openP <= d2_closeP: # D2 양봉
        if d1_openP <= d1_closeP:
            res = "P10"
        elif d2_openP >= d1_closeP:
            res = "S07"
        elif d2_closeP > d1_openP:
            res = "S06"
        elif d2_closeP >= d1_closeP:
            res = "S03"
        else:
            res = "S05"
    elif d2_openP >= d2_closeP: # D2 음봉
        if d1_openP >= d1_closeP:
            res = "M10"
        elif d2_closeP < d1_openP:
            res = "S01"
        elif d2_openP <= d1_closeP:
            res = "S02"
        elif d2_closeP >= d1_closeP:
            res = "S08"
        else:
            res = "S00" 
    else:
        res = "S09"
        
    return res + labellingD0(d210.iloc[2])
stock_list = pd.read_csv('../stockcode.csv')

In [7]:
#today = pd.Timestamp.now()
#today = str(today.year)+str(today.month)+str(today.day)
for stock_index in range(375,486):
    stock_code = str(stock_list['종목코드'][stock_index])
    if len(stock_code) < 6 : 
        stock_code = "0"*(6-len(stock_code)) + stock_code
    stockData = stock.get_market_ohlcv_by_date("20120101", "20201009", stock_code)    # TODO: 종목코드를 csv에서 불러오기
    stockData.col
    umns = pd.Index(["open", "high","low","close","volume"],name=stockData.columns.name)

    stockData['pattern1'] = None
    for i in range(len(stockData)):
        stockData['pattern1'].values[i]  = labellingD0(stockData.iloc[i])

    stockData['pattern2'] = None
    for i in range(1,len(stockData)):
        stockData['pattern2'].values[i] = labellingD1(stockData.iloc[i-1:i+1])

    stockData['pattern3'] = None
    for i in range(2,len(stockData)):
        stockData['pattern3'].values[i] = labellingD2(stockData.iloc[i-2:i+1])
##############################################################################################################################    
    globals()['A' + stock_code + '_1bong'] = stockData["pattern1"]
    
    
    globals()['A' + stock_code + '_bong_list'] = []
    for i in range(0, len(stockData)-3):
        globals()['A' + stock_code + '_1bong' + str(i)] = []
        globals()['A' + stock_code + '_1bong' + str(i)].append("01" + stockData["pattern1"][i])
        globals()['A' + stock_code + '_1bong' + str(i)].append("02" + stockData["pattern1"][i+1])
        globals()['A' + stock_code + '_1bong' + str(i)].append("03" + stockData["pattern1"][i+2])
        globals()['A' + stock_code + '_bong_list'].append(globals()['A' + stock_code + '_1bong'+ str(i)])
    d0 = "02" + stockData["pattern1"][-2]
    d1 = "01" + stockData["pattern1"][-3]
    A = {'P_score' : 0, 'real' : stockData['pattern1'][-1], 'stock_name' : stockData.columns.name, 'stock_code' : stock_code, 
         'P0' : 0, 'P1' : 0, 'M0' : 0, 'M1' : 0, 'K0' : 0}
    for s in globals()['A' + stock_code + '_bong_list'] :
        if d0 in s:
            if d1 in s :
                if "P0" in s[2] :
                    A["P0"] += 1
                elif "P1" in s[2] :
                    A["P1"] += 1
                elif "M0" in s[2] :
                    A["M0"] += 1
                elif "M1" in s[2] :
                    A["M1"] += 1
                elif "K0" in s[2] :
                    A["K0"] += 1
    if (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']) == 0:
        pass
    else :
        A['P_score'] = ((A['P0']+A['P1']) / (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']))
        if A['P_score'] >= 0.5 :
            if A['P1'] < 150:
                if 4 < (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']) :
                    print(A)

{'P_score': 0.6666666666666666, 'real': 'M02', 'stock_name': '에프앤가이드', 'stock_code': '064850', 'P0': 2, 'P1': 0, 'M0': 0, 'M1': 0, 'K0': 1}
{'P_score': 0.532258064516129, 'real': 'M03', 'stock_name': 'S&T모티브', 'stock_code': '064960', 'P0': 33, 'P1': 0, 'M0': 22, 'M1': 0, 'K0': 7}
{'P_score': 0.56, 'real': 'P05', 'stock_name': '지엔코', 'stock_code': '065060', 'P0': 14, 'P1': 0, 'M0': 8, 'M1': 0, 'K0': 3}
{'P_score': 0.5, 'real': 'P01', 'stock_name': '위세아이텍', 'stock_code': '065370', 'P0': 9, 'P1': 1, 'M0': 8, 'M1': 1, 'K0': 1}
{'P_score': 0.9975669099756691, 'real': 'P10', 'stock_name': '에스아이리소스', 'stock_code': '065420', 'P0': 0, 'P1': 410, 'M0': 0, 'M1': 0, 'K0': 1}
{'P_score': 1.0, 'real': 'P10', 'stock_name': '제낙스', 'stock_code': '065620', 'P0': 0, 'P1': 128, 'M0': 0, 'M1': 0, 'K0': 0}
{'P_score': 0.9838056680161943, 'real': 'P10', 'stock_name': '한프', 'stock_code': '066110', 'P0': 0, 'P1': 243, 'M0': 1, 'M1': 2, 'K0': 1}
{'P_score': 0.543859649122807, 'real': 'K01', 'stock_name': '제노텍',

In [20]:
stock_list

,회사명,종목코드
0,동화약품,20
1,KR모터스,40
2,경방,50
3,메리츠화재,60
4,삼양홀딩스,70
...,...,...
2375,코오롱티슈진,950160
2376,JTC,950170
2377,SNK,950180
2378,미투젠,950190


In [ ]:
#today = pd.Timestamp.now()
#today = str(today.year)+str(today.month)+str(today.day)
#range(len(stock_list))
stock_sample = random.sample(range(1, len(stock_list)), 100)
for stock_index in stock_sample:
    stock_code = str(stock_list['종목코드'][stock_index])
    if len(stock_code) < 6 : 
        stock_code = "0"*(6-len(stock_code)) + stock_code
    stockData = stock.get_market_ohlcv_by_date("20120101", "20201020", stock_code)    # TODO: 종목코드를 csv에서 불러오기
    stockData.columns = pd.Index(["open", "high","low","close","volume"],name=stockData.columns.name)
    if stockData["거래량"][-1] == 0:
        pass
    else : 
        stockData['pattern1'] = None
        for i in range(len(stockData)):
            stockData['pattern1'].values[i]  = labellingD0(stockData.iloc[i])

        stockData['pattern2'] = None
        for i in range(1,len(stockData)):
            stockData['pattern2'].values[i] = labellingD1(stockData.iloc[i-1:i+1])

        stockData['pattern3'] = None
        for i in range(2,len(stockData)):
            stockData['pattern3'].values[i] = labellingD2(stockData.iloc[i-2:i+1])
    ##############################################################################################################################    
        #globals()['A' + stock_code + '_1bong'] = stockData["pattern1"]


        bong_list = []
        for i in range(0, len(stockData)-3):
            pattern_list = []
            pattern_list.append("01" + stockData["pattern1"][i])
            pattern_list.append("02" + stockData["pattern1"][i+1])
            pattern_list.append("03" + stockData["pattern1"][i+2])
            bong_list.append(pattern_list)
        d0 = "02" + stockData["pattern1"][-2]
        d1 = "01" + stockData["pattern1"][-3]
        A = {'P_score' : 0, 'real' : stockData['pattern1'][-1], 'stock_name' : stockData.columns.name, 'stock_code' : stock_code, 
             'P0' : 0, 'P1' : 0, 'M0' : 0, 'M1' : 0, 'K0' : 0}
        for bong_order in range(len(bong_list)) :

            if d0 in bong_list[bong_order]:
                if d1 in bong_list[bong_order] :
                    date_weight = math.floor(bong_order/10)
                    if "P0" in bong_list[bong_order][2] :
                        A["P0"] += 1+(0.01*date_weight)
                    elif "P1" in bong_list[bong_order][2] :
                        A["P1"] += 1+(0.01*date_weight)
                    elif "M0" in bong_list[bong_order][2] :
                        A["M0"] += 1+(0.01*date_weight)
                    elif "M1" in bong_list[bong_order][2] :
                        A["M1"] += 1+(0.01*date_weight)
                    elif "K0" in bong_list[bong_order][2] :
                        A["K0"] += 1+(0.01*date_weight)
        if (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']) == 0:
            pass
        else :
            A['P_score'] = round(((A['P0']+A['P1']) / (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']))*100, 2)
            if A['P_score'] >= 50 :
                print(A)

In [ ]:
001470

In [6]:
#stock_sample = random.sample(range(1, len(stock_list)), 100)



stockData = stock.get_market_ohlcv_by_date("20120101", "20201022", "001470")    # TODO: 종목코드를 csv에서 불러오기
stockData.columns = pd.Index(["open", "high","low","close","volume"],name=stockData.columns.name)
if stockData["volume"][-1] == 0:
    pass
else : 
    stockData['pattern1'] = None
    for i in range(len(stockData)):
        stockData['pattern1'].values[i]  = labellingD0(stockData.iloc[i])

    stockData['pattern2'] = None
    for i in range(1,len(stockData)):
        stockData['pattern2'].values[i] = labellingD1(stockData.iloc[i-1:i+1])

    stockData['pattern3'] = None
    for i in range(2,len(stockData)):
        stockData['pattern3'].values[i] = labellingD2(stockData.iloc[i-2:i+1])
##############################################################################################################################    
    #globals()['A' + stock_code + '_1bong'] = stockData["pattern1"]


    bong_list = []
    for i in range(0, len(stockData)-3):
        pattern_list = []
        pattern_list.append("01" + stockData["pattern1"][i])
        pattern_list.append("02" + stockData["pattern1"][i+1])
        pattern_list.append("03" + stockData["pattern1"][i+2])
        bong_list.append(pattern_list)
    d0 = "02" + stockData["pattern1"][-1]
    d1 = "01" + stockData["pattern1"][-2]
    A = {'P_score' : 0, 'real' : stockData['pattern1'][-1], 'stock_name' : stockData.columns.name, 
         'P0' : 0, 'P1' : 0, 'M0' : 0, 'M1' : 0, 'K0' : 0}
    for bong_order in range(len(bong_list)) :

        if d0 in bong_list[bong_order]:
            if d1 in bong_list[bong_order] :
                date_weight = math.floor(bong_order/10)
                if "P0" in bong_list[bong_order][2] :
                    A["P0"] += 1+(0.01*date_weight)
                elif "P1" in bong_list[bong_order][2] :
                    A["P1"] += 1+(0.01*date_weight)
                elif "M0" in bong_list[bong_order][2] :
                    A["M0"] += 1+(0.01*date_weight)
                elif "M1" in bong_list[bong_order][2] :
                    A["M1"] += 1+(0.01*date_weight)
                elif "K0" in bong_list[bong_order][2] :
                    A["K0"] += 1+(0.01*date_weight)
    if (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']) == 0:
        pass
    else :
        A['P_score'] = round(((A['P0']+A['P1']) / (A['P0']+A['P1']+A['M0']+A['M1']+A['K0']))*100, 2)
        print(A)

{'P_score': 50.17, 'real': 'M05', 'stock_name': '삼부토건', 'P0': 4.4, 'P1': 0, 'M0': 4.37, 'M1': 0, 'K0': 0}
